# Axe 2

#### Packages

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import random


### 1) Processing du df composant médical - Emission carbonne (kgCO2e)

In [34]:


def concat_xlsx_from_folder(folder_path: str) -> pd.DataFrame:
    """
    Parcourt récursivement un dossier git et concatène verticalement
    toutes les tables issues des fichiers .xlsx.

    Hypothèses :
    - chaque fichier .xlsx contient une table avec exactement 2 colonnes
    - la première ligne correspond aux labels et est ignorée
    - colonne A : produit
    - colonne B : Emission_kgCO2e_unitaire
    """

    folder = Path(folder_path)
    all_rows = []

    for file in folder.rglob("*.xlsx"):
        df = pd.read_excel(file, header=0)

        df = df.iloc[:, :2]
        df.columns = ["produit", "Emission_kgCO2e_unitaire"]

        filename = file.stem
        if filename.endswith("_parProduit"):
            df["type_de_donnees"] = "parProduit"
        elif filename.endswith("_m2"):
            df["type_de_donnees"] = "m2"
        elif filename.endswith("_parKG"):
            df["type_de_donnees"] = "parKG"
        else:
            continue

        all_rows.append(df)

    if not all_rows:
        return pd.DataFrame(
            columns=["produit", "Emission_kgCO2e_unitaire", "type_de_donnees"]
        )

    df = pd.concat(all_rows, axis=0, ignore_index=True)

    df["Emission_kgCO2e_unitaire"] = (
        df["Emission_kgCO2e_unitaire"]
        .astype(str)
        .str.replace(",", ".", regex=False)
    )
    
    df["Emission_kgCO2e_unitaire"] = pd.to_numeric(
        df["Emission_kgCO2e_unitaire"],
        errors="coerce"
    )
    
    return df


def concat_xlsx_from_folders(list_paths: list[str | Path]) -> pd.DataFrame:
    """
    Concatène verticalement les tables issues de plusieurs dossiers.

    Paramètre
    ----------
    list_paths : list[str | Path]
        Liste de chemins vers des dossiers contenant des fichiers .xlsx

    Retour
    ------
    DataFrame avec les colonnes :
    - produit
    - Emission_kgCO2e_unitaire
    - type_de_donnees
    """

    all_rows = []

    for folder_path in list_paths:
        folder = Path(folder_path)

        if not folder.exists():
            raise FileNotFoundError(f"Dossier introuvable : {folder}")

        xlsx_files = list(folder.rglob("*.xlsx"))
        if not xlsx_files:
            raise ValueError(f"Aucun fichier .xlsx trouvé dans {folder}")

        for file in xlsx_files:
            df = pd.read_excel(file)

            df = df.iloc[:, :2]
            df.columns = ["produit", "Emission_kgCO2e_unitaire"]

            name = file.stem
            if name.endswith("_parProduit"):
                df["type_de_donnees"] = "parProduit"
            elif name.endswith("_m2"):
                df["type_de_donnees"] = "m2"
            elif name.endswith("_parKG"):
                df["type_de_donnees"] = "parKG"
            else:
                continue

            all_rows.append(df)

    if not all_rows:
        return pd.DataFrame(
            columns=["produit", "Emission_kgCO2e_unitaire", "type_de_donnees"]
        )

    df = pd.concat(all_rows, axis=0, ignore_index=True)
    
    df["Emission_kgCO2e_unitaire"] = (
        df["Emission_kgCO2e_unitaire"]
        .astype(str)
        .str.replace(",", ".", regex=False)
    )

    df["Emission_kgCO2e_unitaire"] = pd.to_numeric(
        df["Emission_kgCO2e_unitaire"],
        errors="coerce"
    )
    
    return df



def random_value_dict(
    df: pd.DataFrame,
    nom_col: str = "produit",
    nom_to_ignore: list | None = None,
    min_value: float = 1.0,
    max_value: float = 3000.0,
) -> dict:
    """
    Construit un dictionnaire :
    - clés : valeurs uniques de df[nom_col]
    - valeurs : nombre aléatoire strictement > 1
      tiré dans [min_value, max_value]
    """

    if nom_to_ignore is None:
        nom_to_ignore = []

    if min_value <= 1:
        min_value = 1.000001

    uniques = df[nom_col].dropna().unique()

    return {
        val: random.uniform(min_value, max_value)
        for val in uniques
        if val not in nom_to_ignore
    }


def compute_emission_hopital(
    df: pd.DataFrame,
    type_de_donnees: str = "type_de_donnees",
    dict_m2: dict = {"pansements composites": (3 * 10**2, 10)},
    dict_parKG: dict = {"instrument usage unique": 1000, "complement alimentaire":10000},
    dict_nb_parProduit: dict | None = None,
) -> pd.DataFrame:
    """
    Ajoute la colonne Emission_carbonne_total_des_produits_kgCO2e selon
    le type de données associé à chaque produit.
    """

    if dict_nb_parProduit is None:
        dict_nb_parProduit = {}

    def compute_row(row):
        produit = row["produit"]
        emission_unit = row["Emission_kgCO2e_unitaire"]
        t = row[type_de_donnees]

        if t == "parProduit":
            return emission_unit * dict_nb_parProduit.get(produit, np.nan)

        if t == "m2":
            longueur, largeur = dict_m2.get(produit, (np.nan, np.nan))
            return emission_unit * longueur * largeur

        if t == "parKG":
            return emission_unit * dict_parKG.get(produit, np.nan)

        return np.nan

    df = df.copy()
    df["Emission_carbonne_total_des_produits_kgCO2e"] = df.apply(compute_row, axis=1)

    return df

# =========================
# 3. Construction du DataFrame
# =========================

# extract_path = "sujets/chu/Axe_2/Axe_2_bdd"
extract_path = "C:/Users/jerem/Documents/GitHub/datachallenge2026/sujets/chu/Axe_2/Axe_2_bdd"
# paths = [
#     r"sujets\chu\Axe_2\Axe_2_bdd-20260117T004817Z-1-001\Axe_2_bdd",
#     r"sujets\chu\Axe_2\autre_dossier"
# ]

df_concat = concat_xlsx_from_folder(extract_path)
# df_concat = concat_xlsx_from_folders(paths)

print(df_concat.head())

# =========================
# 4. Dictionnaires exemples
# =========================

dict_nb_parProduit = random_value_dict(df_concat)

print(dict_nb_parProduit)

# =========================
# 5. Calcul des émissions
# =========================

df_final = compute_emission_hopital(
    df_concat,
    dict_nb_parProduit=dict_nb_parProduit
)

print(df_final.head())



                   produit  Emission_kgCO2e_unitaire type_de_donnees
0    pansements composites                    0.9400              m2
1           Sonde urinaire                   51.9034      parProduit
2  Set de sondage urinaire                   89.4064      parProduit
3      Collecteur de jambe                  250.7186      parProduit
4       Collecteur de nuit                  676.6522      parProduit
{'pansements composites': 2880.7506438973483, 'Sonde urinaire': 2.3058391190603773, 'Set de sondage urinaire': 1301.0278534529064, 'Collecteur de jambe': 1871.2095479103848, 'Collecteur de nuit': 2265.768898794339, 'Etuis péniens': 1088.7167868279912, 'Poche pour stomie': 3.131259971503482, 'Support pour stomie': 2959.0238083770532, 'Changes complets': 2375.889528275952, 'Slips absorbants': 250.72891706628525, 'Protections absorbantes': 1887.063988630245, 'Couches droites': 1329.456550312355, 'Alèses': 1383.9846297425656, 'pansement': 2495.056711895948, 'uteruscopes': 2579.211406

In [35]:
# Doublons sur une variable particulière (ex: 'email')
colonne = 'produit'

# Trouver les valeurs dupliquées dans cette colonne
valeurs_doublons = df_final[df_final.duplicated(subset=[colonne], keep=False)]

# Afficher les doublons triés pour mieux voir
doublons_tries = valeurs_doublons.sort_values(colonne)
print(f"Doublons sur la colonne '{colonne}' :")
print(doublons_tries)

# Voir les valeurs qui se répètent
comptage = df_final[colonne].value_counts()
valeurs_repetees = comptage[comptage > 1]
print(f"\nValeurs répétées dans '{colonne}' :")
print(valeurs_repetees)


Doublons sur la colonne 'produit' :
Empty DataFrame
Columns: [produit, Emission_kgCO2e_unitaire, type_de_donnees, Emission_carbonne_total_des_produits_kgCO2e]
Index: []

Valeurs répétées dans 'produit' :
Series([], Name: count, dtype: int64)


#### 1.1) Obtention du dataframe final et exportation

In [36]:
colonnes_a_conserver = [
    "produit",
    "Emission_kgCO2e_unitaire",
    "Emission_carbonne_total_des_produits_kgCO2e"
]

df_export = df_final[colonnes_a_conserver].copy()

# =========================
# Chemin de sortie
# =========================

output_path = Path(r"results\df_composant_medical_emissions_carbones.xlsx")

# Création du dossier si besoin
output_path.parent.mkdir(parents=True, exist_ok=True)

# =========================
# Export Excel
# =========================

df_export.to_excel(output_path, index=False)